In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import shap
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def carregar_dados_ticker(ticker, pasta_dados):
    """Carrega dados de um ticker com foco nas features macro e micro"""
    arquivo = f"{pasta_dados}/{ticker}_dados_macro_micro.csv"
    
    if not os.path.exists(arquivo):
        print(f"Arquivo não encontrado: {arquivo}")
        return None, None, None
    
    # Carregar dados
    try:
        df = pd.read_csv(arquivo, index_col=0)
        df.index = pd.to_datetime(df.index)
    except Exception as e:
        print(f"Erro ao carregar {arquivo}: {e}")
        return None, None, None
    
    # Identificar target
    target_col = None
    if 'Close_Target' in df.columns:
        target_col = 'Close_Target'
    elif 'Preço' in df.columns:
        target_col = 'Preço'
    else:
        print(f"Erro: Nenhuma coluna target encontrada para {ticker}")
        return None, None, None
    
    # Separar features em macro e micro
    macro_cols = [col for col in df.columns if col in ['TaxaCambio', 'Selic', 'PIB', 'IPCA']]
    micro_cols = [col for col in df.columns if col in ['ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP']]
    price_col = [col for col in df.columns if col in ['Close_Feature', 'Preço_anterior']]
    
    # Todos os conjuntos de features
    feature_sets = {
        'price_only': price_col,
        'macro_only': macro_cols,
        'micro_only': micro_cols,
        'price_macro': price_col + macro_cols,
        'price_micro': price_col + micro_cols,
        'macro_micro': macro_cols + micro_cols,
        'all': price_col + macro_cols + micro_cols
    }
    
    # Filtrar conjuntos vazios e remover duplicatas
    feature_sets = {k: list(set(v)) for k, v in feature_sets.items() if v}
    
    return df, feature_sets, target_col

def construir_modelo_simples(input_shape):
    """Constrói um modelo simples para análise de importância de features"""
    model = keras.Sequential([
        layers.Dense(64, activation='relu', input_shape=input_shape),
        layers.Dropout(0.2),
        layers.Dense(32, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

def calcular_importancia_shap(model, X_background, X_test):
    """Calcula valores SHAP para entender importância de features"""
    explainer = shap.DeepExplainer(model, X_background)
    shap_values = explainer.shap_values(X_test)
    return shap_values[0]  # Para modelos com uma saída

def comparar_conjuntos_features(ticker, df, feature_sets, target_col):
    """Compara desempenho de diferentes conjuntos de features"""
    # Dividir dados (treino/teste)
    split_date = "2023-01-01"
    df_train = df.loc[:split_date].copy()
    df_test = df.loc[split_date:].copy()
    
    if df_train.empty or df_test.empty:
        print(f"Erro: Divisão treino/teste resultou em conjunto vazio para {ticker}")
        return None
    
    # Resultados
    resultados = {}
    importancias = {}
    
    for set_name, features in feature_sets.items():
        print(f"\nAvaliando {ticker} com conjunto {set_name}: {features}")
        
        # Verificar se temos dados suficientes
        if any(col not in df.columns for col in features):
            print(f"  Pulando {set_name}: faltam colunas")
            continue
            
        # Preparar dados
        X_train = df_train[features].values
        y_train = df_train[target_col].values
        
        X_test = df_test[features].values
        y_test = df_test[target_col].values
        
        # Escalonar
        scaler_X = StandardScaler()
        scaler_y = StandardScaler()
        
        X_train_scaled = scaler_X.fit_transform(X_train)
        y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
        
        X_test_scaled = scaler_X.transform(X_test)
        y_test_scaled = scaler_y.transform(y_test.reshape(-1, 1)).flatten()
        
        # Construir e treinar modelo
        input_shape = (X_train_scaled.shape[1],)
        model = construir_modelo_simples(input_shape)
        
        early_stopping = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=10, restore_best_weights=True
        )
        
        # Treinar com validação
        history = model.fit(
            X_train_scaled, y_train_scaled,
            validation_split=0.2,
            epochs=100,
            batch_size=32,
            callbacks=[early_stopping],
            verbose=0
        )
        
        # Avaliar
        y_pred_scaled = model.predict(X_test_scaled)
        y_pred = scaler_y.inverse_transform(y_pred_scaled).flatten()
        
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        var_real = np.std(y_test) / np.mean(y_test) if np.mean(y_test) != 0 else 0
        var_pred = np.std(y_pred) / np.mean(y_pred) if np.mean(y_pred) != 0 else 0
        
        resultados[set_name] = {
            'RMSE': rmse,
            'Var_Real': var_real,
            'Var_Pred': var_pred,
            'Prop_Var': var_pred / var_real if var_real != 0 else 0
        }
        
        print(f"  RMSE: {rmse:.4f}, Var Pred/Real: {resultados[set_name]['Prop_Var']:.4f}")
        
        # Calcular importância das features (apenas para conjunto completo)
        if set_name == 'all' and len(X_train_scaled) > 0:
            try:
                # Usar subset para SHAP (para economizar tempo)
                background_indices = np.random.choice(X_train_scaled.shape[0], 
                                                     min(100, X_train_scaled.shape[0]), 
                                                     replace=False)
                X_background = X_train_scaled[background_indices]
                
                test_indices = np.random.choice(X_test_scaled.shape[0], 
                                              min(50, X_test_scaled.shape[0]), 
                                              replace=False)
                X_test_sample = X_test_scaled[test_indices]
                
                shap_values = calcular_importancia_shap(model, X_background, X_test_sample)
                
                # Média absoluta dos valores SHAP por feature
                importancias_medias = np.abs(shap_values).mean(axis=0)
                importancias = {features[i]: importancias_medias[i] for i in range(len(features))}
                print("  Importância de features (SHAP):")
                for feat, imp in sorted(importancias.items(), key=lambda x: x[1], reverse=True):
                    print(f"    {feat}: {imp:.4f}")
            except Exception as e:
                print(f"  Erro ao calcular SHAP: {e}")
    
    return resultados, importancias

def plotar_resultados(ticker, resultados, importancias, output_dir):
    """Plota resultados da comparação de conjuntos de features"""
    os.makedirs(output_dir, exist_ok=True)
    
    if not resultados:
        print(f"Sem resultados para plotar para {ticker}")
        return
    
    # 1. Gráfico de RMSE por conjunto de features
    plt.figure(figsize=(10, 6))
    sets = list(resultados.keys())
    rmse_values = [resultados[s]['RMSE'] for s in sets]
    
    # Ordenar do melhor para o pior
    sorted_indices = np.argsort(rmse_values)
    sorted_sets = [sets[i] for i in sorted_indices]
    sorted_rmse = [rmse_values[i] for i in sorted_indices]
    
    # Plotar barras
    bars = plt.bar(sorted_sets, sorted_rmse)
    
    # Colorir barras por tipo
    colors = {'price_only': 'gray', 'macro_only': 'blue', 'micro_only': 'green', 
              'price_macro': 'lightblue', 'price_micro': 'lightgreen', 
              'macro_micro': 'purple', 'all': 'orange'}
    
    for i, set_name in enumerate(sorted_sets):
        if set_name in colors:
            bars[i].set_color(colors[set_name])
    
    plt.title(f'RMSE por Conjunto de Features - {ticker}')
    plt.ylabel('RMSE (menor é melhor)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{ticker}_rmse_by_feature_set.png")
    plt.close()
    
    # 2. Gráfico de proporção de variação (quanto as previsões capturam da variação real)
    plt.figure(figsize=(10, 6))
    prop_var_values = [resultados[s]['Prop_Var'] for s in sets]
    
    # Ordenar do melhor (maior proporção) para o pior
    sorted_indices_var = np.argsort(prop_var_values)[::-1]  # Inverter para ter descendente
    sorted_sets_var = [sets[i] for i in sorted_indices_var]
    sorted_prop_var = [prop_var_values[i] for i in sorted_indices_var]
    
    # Plotar barras
    bars_var = plt.bar(sorted_sets_var, sorted_prop_var)
    
    # Colorir barras por tipo
    for i, set_name in enumerate(sorted_sets_var):
        if set_name in colors:
            bars_var[i].set_color(colors[set_name])
    
    # Linha de referência: 100% da variação real
    plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.6)
    
    plt.title(f'Proporção de Variação Capturada - {ticker}')
    plt.ylabel('Variação Prevista / Variação Real')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/{ticker}_variation_proportion_by_feature_set.png")
    plt.close()
    
    # 3. Gráfico de importância das features (SHAP)
    if importancias:
        plt.figure(figsize=(10, 6))
        features = list(importancias.keys())
        importance_values = list(importancias.values())
        
        # Ordenar por importância
        sorted_indices_imp = np.argsort(importance_values)
        sorted_features = [features[i] for i in sorted_indices_imp]
        sorted_importance = [importance_values[i] for i in sorted_indices_imp]
        
        # Definir cores por tipo de feature
        bar_colors = []
        for feature in sorted_features:
            if feature in ['TaxaCambio', 'Selic', 'PIB', 'IPCA']:
                bar_colors.append('blue')  # Macro
            elif feature in ['ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP']:
                bar_colors.append('green')  # Micro
            else:
                bar_colors.append('gray')  # Preço
        
        # Plotar barras horizontais
        bars_imp = plt.barh(sorted_features, sorted_importance, color=bar_colors)
        
        plt.title(f'Importância de Features (SHAP) - {ticker}')
        plt.xlabel('Importância Relativa')
        plt.tight_layout()
        plt.savefig(f"{output_dir}/{ticker}_feature_importance_shap.png")
        plt.close()

def analisar_tickers():
    """Analisa a importância de features macro e micro para cada ticker"""
    pasta_dados = r"C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dados_unificados"
    output_dir = r"C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\comparacao_macromicro"
    os.makedirs(output_dir, exist_ok=True)
    
    tickers = ["BEEF3.SA", "PETR4.SA", "SOJA3.SA", "GGBR3.SA", "CSNA3.SA", 
               "VALE3.SA", "JBSS3.SA", "BRFS3.SA", "SUZB3.SA"]
    
    resultados_globais = []
    
    for ticker in tickers:
        print(f"\n====== Analisando {ticker} ======")
        df, feature_sets, target_col = carregar_dados_ticker(ticker, pasta_dados)
        
        if df is None:
            print(f"Pulando {ticker} - dados não disponíveis")
            continue
            
        # Analisar e comparar conjuntos de features
        resultados, importancias = comparar_conjuntos_features(ticker, df, feature_sets, target_col)
        
        if resultados:
            # Armazenar resultados globais
            for set_name, metrics in resultados.items():
                resultados_globais.append({
                    'Ticker': ticker,
                    'Feature_Set': set_name, 
                    'RMSE': metrics['RMSE'],
                    'Prop_Var': metrics['Prop_Var']
                })
            
            # Plotar resultados específicos do ticker
            plotar_resultados(ticker, resultados, importancias, output_dir)
    
    # Salvar resultados consolidados
    if resultados_globais:
        df_resultados = pd.DataFrame(resultados_globais)
        df_resultados.to_csv(f"{output_dir}/resultados_consolidados_macromicro.csv", index=False)
        
        # Plotar comparação geral entre tipos de features
        plt.figure(figsize=(14, 8))
        
        # Calcular mediana de RMSE por conjunto de features
        mediana_por_conjunto = df_resultados.groupby('Feature_Set')['RMSE'].median()
        mediana_por_conjunto = mediana_por_conjunto.sort_values()
        
        plt.bar(mediana_por_conjunto.index, mediana_por_conjunto.values)
        plt.title('RMSE Mediano por Conjunto de Features (Todos Tickers)')
        plt.ylabel('RMSE Mediano (menor é melhor)')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f"{output_dir}/comparacao_geral_rmse_por_feature_set.png")
        plt.close()

if __name__ == "__main__":
    print("Iniciando análise de importância de features macro/micro...")
    
    # Reduzir verbosidade do TensorFlow
    tf.get_logger().setLevel('ERROR')
    
    analisar_tickers()
    
    print("Análise concluída!")


Iniciando análise de importância de features macro/micro...

====== Analisando BEEF3.SA ======

Avaliando BEEF3.SA com conjunto price_only: ['Close_Feature', 'Preço_anterior']


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
  RMSE: 1.8727, Var Pred/Real: 0.0000

Avaliando BEEF3.SA com conjunto macro_only: ['IPCA', 'TaxaCambio', 'Selic', 'PIB']


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
  RMSE: 4.0686, Var Pred/Real: 0.2434

Avaliando BEEF3.SA com conjunto micro_only: ['P/L', 'ROA', 'Margem Líquida', 'ROE', 'VP']


...

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
  RMSE: 5.2175, Var Pred/Real: 0.3084
  Erro ao calcular SHAP: index 1 is out of bounds for axis 0 with size 1


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\shap\explainers\_deep\deep_tf.py:94: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn(



====== Analisando BRFS3.SA ======

Avaliando BRFS3.SA com conjunto price_only: ['Preço_anterior']


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 